In [1]:
%reload_ext Cython

In [2]:
import training
import importlib
importlib.reload(training)
train_data = training.TrainingData_PQR("good_games.txt",80)

50 out of 80


In [65]:
import cProfile
import tensorflow as tf
import os
import re
import chess
import test
import boardclass
#import training
import importlib
import numpy as np
importlib.reload(boardclass)
importlib.reload(test)
importlib.reload(training)

<module 'training' from '/Users/dawit/Documents/Datalogi/MachineLearning_Projects/Chess/src/training.py'>

In [66]:
board = boardclass.ChessBoard(-1)
board.train_model(80,train_data,1)


#board.restore_model()
#
#score,move = board.return_best_move(1)
####board.close_session()
###
#print(score,move)
##tf.reset_default_graph()

score P: [[ 71.17446136]]
score Q: [[ 108.17390442]]
score R: [[-33.70529938]]
score P: [[ 53.91858292]]
score Q: [[ 90.52735138]]
score R: [[-49.33240509]]
score P: [[ 37.41042709]]
score Q: [[ 73.48995972]]
score R: [[-64.11029053]]
score P: [[ 22.10235786]]
score Q: [[ 57.49580765]]
score R: [[-77.57301331]]
score P: [[ 8.53649807]]
score Q: [[ 43.07761765]]
score R: [[-89.17353058]]
score P: [[-2.75610471]]
score Q: [[ 30.76354408]]
score R: [[-98.3970108]]
score P: [[-11.41149426]]
score Q: [[ 20.9348259]]
score R: [[-104.91848755]]
score P: [[-17.32005501]]
score Q: [[ 13.73378944]]
score R: [[-108.67835236]]
score P: [[-20.60741615]]
score Q: [[ 9.07090473]]
score R: [[-109.8462677]]
score P: [[-21.55430603]]
score Q: [[ 6.6957655]]
score R: [[-108.73182678]]
score P: [[-20.51544762]]
score Q: [[ 6.27591515]]
score R: [[-105.70246124]]
score P: [[-17.86582375]]
score Q: [[ 7.45062542]]
score R: [[-101.13292694]]
score P: [[-13.97441959]]
score Q: [[ 9.85967922]]
score R: [[-95.3

In [ ]:
for i in range(20):
    score,move = board.return_best_move(1)
    print("Score: {}".format(score))
    board.make_move(move)
    board.show_board()

In [6]:

importlib.reload(test)
%run testcases.py

.................
----------------------------------------------------------------------
Ran 17 tests in 0.030s

OK
